In [2]:
import os
os.chdir("..")
import sys
sys.path.append("process")

In [3]:
from cfps_shell import *
from functools import partial
pd.options.mode.chained_assignment = None 

幸福程度 （0 - 10）

In [31]:
def get_data_for(description,year,table, provcdid, col, coefficient=1, extra_less_than = False):
    cols = [provcdid,col]
    if extra_less_than:
        cols.append('age')
    data_child = cfps[year][table].data[cols]
    data_child[provcdid] = data_child[provcdid].apply(lambda x: "NaN" if pd.isna(x) else get_prov(x))
    valids = data_child[data_child[col]>0]
    if extra_less_than:
        valids = valids[valids["age"]<=16]
    ans = coefficient*valids.groupby(provcdid).mean()
    data = [ {"name": k, "value": v } for k,v in ans.to_dict()[col].items()]
    return f"""{{
    title: {{
        text: "{description}({year})",
    }},
    series: [
        {{
            name: '{description}',
            type: 'map',
            map: 'china',
            data: {data}
        }}
    ]
}},"""
get_data_for = partial(get_data_for, "儿童幸福程度")

In [32]:
print(get_data_for(2018, "person", "provcd18", "qm2016", 1, True))

{
    title: {
        text: "儿童幸福程度(2018)",
    },
    series: [
        {
            name: '儿童幸福程度',
            type: 'map',
            map: 'china',
            data: [{'name': '上海市', 'value': 8.402777777777779}, {'name': '云南省', 'value': 8.198019801980198}, {'name': '北京市', 'value': 9.5}, {'name': '吉林省', 'value': 8.233333333333333}, {'name': '四川省', 'value': 8.042780748663102}, {'name': '天津市', 'value': 8.25}, {'name': '宁夏回族自治区', 'value': 10.0}, {'name': '安徽省', 'value': 8.081632653061224}, {'name': '山东省', 'value': 8.634615384615385}, {'name': '山西省', 'value': 8.31}, {'name': '广东省', 'value': 7.840677966101695}, {'name': '广西壮族自治区', 'value': 7.584269662921348}, {'name': '新疆维吾尔自治区', 'value': 9.333333333333334}, {'name': '江苏省', 'value': 7.7073170731707314}, {'name': '江西省', 'value': 7.7924528301886795}, {'name': '河北省', 'value': 8.48936170212766}, {'name': '河南省', 'value': 8.394009216589861}, {'name': '浙江省', 'value': 8.0}, {'name': '湖北省', 'value': 8.470588235294118}, {'name': '湖南省', 'value':

In [33]:
print(''.join(map(lambda x: get_data_for(*x), [
    (2010, "child", "provcd", "wm302",2),
    (2012, "child", "provcd", "wm302_b_2"),
    (2014, "child", "provcd14", "wm302"),
    (2016, "child", "provcd16", "qm2014"),
    (2018, "person", "provcd18", "qm2016", 1, True),
])))

{
    title: {
        text: "儿童幸福程度(2010)",
    },
    series: [
        {
            name: '儿童幸福程度',
            type: 'map',
            map: 'china',
            data: [{'name': '上海市', 'value': 8.62962962962963}, {'name': '云南省', 'value': 8.693877551020408}, {'name': '北京市', 'value': 8.222222222222221}, {'name': '吉林省', 'value': 8.976744186046512}, {'name': '四川省', 'value': 8.239520958083832}, {'name': '天津市', 'value': 9.142857142857142}, {'name': '安徽省', 'value': 8.046511627906977}, {'name': '山东省', 'value': 8.714285714285714}, {'name': '山西省', 'value': 8.345238095238095}, {'name': '广东省', 'value': 7.797520661157025}, {'name': '广西壮族自治区', 'value': 8.204081632653061}, {'name': '江苏省', 'value': 8.448275862068966}, {'name': '江西省', 'value': 8.023529411764706}, {'name': '河北省', 'value': 8.867647058823529}, {'name': '河南省', 'value': 8.529680365296803}, {'name': '浙江省', 'value': 8.714285714285714}, {'name': '湖北省', 'value': 8.555555555555555}, {'name': '湖南省', 'value': 8.057692307692308}, {'name': '甘肃省

In [37]:
def get_data_for_urban_rural(year,table, col, coefficient=1, extra_less_than = False):
    cols = [col]
    if extra_less_than:
        cols.append('age')
    data_child_urban = cfps[year][table].urban[cols]
    data_child_rural = cfps[year][table].rural[cols]
    valids_urban = data_child_urban[data_child_urban[col]>0]
    valids_rural = data_child_rural[data_child_rural[col]>0]
    if extra_less_than:
        valids_urban = valids_urban[valids_urban["age"]<=16]
        valids_rural = valids_rural[valids_rural["age"]<=16]
    ans_urban = coefficient * valids_urban[col].mean()
    ans_rural = coefficient * valids_rural[col].mean()
    return [year, ans_urban, ans_rural]

In [38]:
print(list(map(lambda x: get_data_for_urban_rural(*x), [
    (2010, "child", "wm302",2),
    (2012, "child", "wm302_b_2"),
    (2014, "child", "wm302"),
    (2016, "child", "qm2014"),
    (2018, "person","qm2016", 1, True),
])))

[[2010, 8.50229709035222, 8.243720930232557], [2012, 7.967149758454107, 7.757125154894672], [2014, 8.396370582617001, 8.165886287625417], [2016, 8.478170478170478, 8.138198757763975], [2018, 8.197123519458545, 8.108892921960072]]
